<a href="https://colab.research.google.com/github/KimJisanER/Dacon/blob/main/Dacon_Metabolism_0913_beta_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install mordred

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import Descriptors
from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Descriptor 함수 정의

In [ ]:
#1
def getMolDescriptors(mol, missingVal=None):
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

#2
def extract_fragments(smiles, hybridization_type=None, exclude_ring=False):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        hybridization = atom.GetHybridization()
        if (hybridization_type is None or hybridization == hybridization_type) and (not exclude_ring or not atom.IsInRing()):
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

def count_fragments(full, column_prefix, hybridization_type=None, exclude_ring=False):
    fragment_counts = {}
    for index, row in full.iterrows():
        smiles = row['SMILES']
        fragments = extract_fragments(smiles, hybridization_type, exclude_ring)

        fragment_count = {}
        for idx, fragment in enumerate(fragments):
            fragment_smiles = Chem.MolToSmiles(fragment)
            if fragment_smiles in fragment_count:
                fragment_count[fragment_smiles] += 1
            else:
                fragment_count[fragment_smiles] = 1

        fragment_counts[index] = fragment_count

    result_data = []
    for index, counts in fragment_counts.items():
        counts['SMILES'] = full.loc[index, 'SMILES']
        result_data.append(counts)

    result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
    result_df.columns = [f'{column_prefix}_{column}' for column in result_df.columns]

    return result_df

#3
def calculate_molecular_properties(full):
    def calculate_stereocenters(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)
        return len(chiral_centers)

    def calculate_aromatic_proportion(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        aromatic_atoms = [atom for atom in mol.GetAtoms() if atom.GetIsAromatic()]
        total_atoms = mol.GetNumAtoms()
        if total_atoms == 0:
            return None
        return len(aromatic_atoms) / total_atoms

    def calculate_formal_charge(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        return Chem.rdmolops.GetFormalCharge(mol)

    properties_data = []

    for index, row in full.iterrows():
        smiles = row['SMILES']
        stereocenters = calculate_stereocenters(smiles)
        proportion = calculate_aromatic_proportion(smiles)
        charge = calculate_formal_charge(smiles)

        if stereocenters is not None and proportion is not None and charge is not None:
            properties_data.append({
                "stereocenters": stereocenters,
                "aromatic_proportion": proportion,
                "formal_charge": charge
            })

    properties_df = pd.DataFrame(properties_data)

    return properties_df

#데이터 가공

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])

#ChEMBL data
MLM = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train_aug_MLM_0906.csv")
HLM = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train_aug_HLM_0906.csv")

test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])

full = pd.concat([train, test, MLM, HLM], axis = 0).reset_index(drop=True)
full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)

In [ ]:
duplicates = full[full.duplicated(subset='SMILES', keep=False)].sort_values(by='SMILES').reset_index(drop=True)
prop = calculate_molecular_properties(duplicates).reset_index(drop=True)
dupli_df = pd.concat([duplicates, prop], axis=1)[['SMILES', 'MLM', 'HLM', 'stereocenters']]

# 'stereocenters' 컬럼 값이 0인 데이터 필터링
filtered_df = dupli_df[dupli_df['stereocenters'] == 0]

# 'SMILES'를 기준으로 그룹화하고 'MLM'과 'HLM'의 평균 계산
mean_mlm = filtered_df.groupby('SMILES')['MLM'].mean().reset_index()
mean_hlm = filtered_df.groupby('SMILES')['HLM'].mean().reset_index()

# 원본 DataFrame을 수정
for index, row in mean_mlm.iterrows():
    smiles = row['SMILES']
    mlm_mean = row['MLM']
    hlm_mean = mean_hlm[mean_hlm['SMILES'] == smiles]['HLM'].values[0]
    dupli_df.loc[dupli_df['SMILES'] == smiles, 'MLM'] = mlm_mean
    dupli_df.loc[dupli_df['SMILES'] == smiles, 'HLM'] = hlm_mean

mean_duplicate = dupli_df.drop_duplicates().reset_index(drop=True)

# 입체이성질체 없는 경우 동일 분자의 실험값들을 mean으로 처리.
for index, row in mean_duplicate.iterrows():
    s_value = row['SMILES']
    matching_indices = full[full['SMILES'] == s_value].index
    for full_index in matching_indices:
        full.at[full_index, 'MLM'] = row['MLM']
        full.at[full_index, 'HLM'] = row['HLM']

filtered_full = full.drop_duplicates(subset=['SMILES', 'HLM', 'MLM'], keep='first').reset_index(drop=True)

In [ ]:
full

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.25900,117.37,<rdkit.Chem.rdchem.Mol object at 0x7be41781e960>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.17200,73.47,<rdkit.Chem.rdchem.Mol object at 0x7be41781e9d0>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.58500,62.45,<rdkit.Chem.rdchem.Mol object at 0x7be41781ea40>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.47500,92.60,<rdkit.Chem.rdchem.Mol object at 0x7be41781eab0>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.33700,42.43,<rdkit.Chem.rdchem.Mol object at 0x7be41781eb20>
...,...,...,...,...,...,...,...,...,...,...,...,...
6245,CHEMBL3286701,Cc1[nH]c2ccccc2c1CCNC(=O)c1ccc(N(C)C)cc1,0.000,17.000,3.510,321.420,2,2,5,3.51482,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6dc0>
6246,CHEMBL215387,O=C(O)CCCCCCCCCCCNC(=O)NC12CC3CC(CC(C3)C1)C2,NaN,83.500,5.240,392.580,2,3,13,5.24000,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6e30>
6247,CHEMBL242459,O=C(NC12CC3CC(CC(C3)C1)C2)N[C@H]1CC[C@H](Oc2cc...,NaN,83.000,4.340,412.530,3,3,5,4.34280,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6ea0>
6248,CHEMBL3735279,CCOC(=O)c1nc(C)c2c(c1N)C(=O)N(Cc1ccccc1)C2=O,NaN,14.000,1.950,339.350,6,1,4,1.94512,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6f10>


In [ ]:
filtered_full

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.25900,117.37,<rdkit.Chem.rdchem.Mol object at 0x7be41781e960>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.17200,73.47,<rdkit.Chem.rdchem.Mol object at 0x7be41781e9d0>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.58500,62.45,<rdkit.Chem.rdchem.Mol object at 0x7be41781ea40>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.47500,92.60,<rdkit.Chem.rdchem.Mol object at 0x7be41781eab0>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.33700,42.43,<rdkit.Chem.rdchem.Mol object at 0x7be41781eb20>
...,...,...,...,...,...,...,...,...,...,...,...,...
5742,CHEMBL3810411,Cc1cc(CCC#N)cc(C)c1Oc1cc(Nc2ccc(C#N)cc2)c(N)cc...,NaN,78.400,6.000,451.570,6,3,9,6.00140,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6c00>
5743,CHEMBL3092907,Cc1nc(N2CCN(C)CC2)c2nc(-c3ccccc3Cl)n(CCO)c2n1.Cl,NaN,13.000,2.200,423.350,7,1,4,2.62102,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6c70>
5744,CHEMBL2177757,O=C(Cc1ccccc1)Nc1nnc(CCSCCc2nnc(NC(=O)Cc3ccccc...,NaN,56.000,4.270,524.700,9,2,12,4.27040,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6ce0>
5745,CHEMBL3735279,CCOC(=O)c1nc(C)c2c(c1N)C(=O)N(Cc1ccccc1)C2=O,NaN,14.000,1.950,339.350,6,1,4,1.94512,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6f10>


In [ ]:
# Mordred
calc = Calculator(descriptors, ignore_3D=True)
mord_desc_df = calc.pandas([Chem.MolFromSmiles(x) for x in filtered_full.SMILES])
mord_desc_df.columns = ['mord_'+ column for column in mord_desc_df.columns]
# object 형식 열 삭제
mord_desc_df.drop(columns=mord_desc_df.select_dtypes(include=['object']).columns, inplace=True)

# bool 형식 열을 0과 1로 인코딩
bool_columns = mord_desc_df.select_dtypes(include=['bool']).columns
mord_desc_df[bool_columns] = mord_desc_df[bool_columns].astype(int)

print(mord_desc_df.head())

  4%|▍         | 249/5747 [04:41<3:56:28,  2.58s/it]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 95%|█████████▍| 5434/5747 [1:46:18<06:09,  1.18s/it]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 5747/5747 [1:52:50<00:00,  1.18s/it]


    mord_ABC  mord_ABCGG  mord_nAcid  mord_nBase  mord_nAromAtom  \
0  21.379612   17.449011           0           0              16   
1  16.539255   14.049653           0           0              11   
2  17.475469   13.660693           2           1              13   
3  27.857311   20.034364           0           1              17   
4  15.722758   12.817176           0           0              12   

   mord_nAromBond  mord_nAtom  mord_nHeavyAtom  mord_nSpiro  mord_nBridgehead  \
0              16          52               28            0                 0   
1              11          40               21            0                 0   
2              15          41               22            0                 0   
3              17          69               35            0                 0   
4              12          36               20            0                 0   

   ...  mord_SRW09  mord_SRW10  mord_TSRW10     mord_MW  mord_AMW  mord_WPath  \
0  ...    7.390799   10

In [ ]:
from rdkit import DataStructs

allDescrs = [getMolDescriptors(m) for m in filtered_full['Molecule']]
full_Descrs = pd.DataFrame(allDescrs)

In [ ]:
result_df_sp3 = count_fragments(filtered_full, 'sp3', Chem.HybridizationType.SP3)
result_df_non_ring_sp3 = count_fragments(filtered_full, 'sp3', Chem.HybridizationType.SP3, exclude_ring=True)
result_df_sp2 = count_fragments(filtered_full, 'sp2', Chem.HybridizationType.SP2)
result_df_non_ring_sp2 = count_fragments(filtered_full, 'non_ring_sp2', Chem.HybridizationType.SP2, exclude_ring=True)
result_df_sp = count_fragments(filtered_full, 'sp', Chem.HybridizationType.SP)

prop_df = calculate_molecular_properties(filtered_full)

In [ ]:
result_df_non_ring_sp3

,sp3_CC,sp3_CCO,sp3_cCN,sp3_Cc,sp3_CCC,sp3_CCN,sp3_cS(N)(=O)=O,sp3_CO,sp3_cF,sp3_CN,...,sp3_NP(=O)(O)O,sp3_CC(C)[N+],sp3_C[C@@H](C)S,sp3_O=P([O-])(O)O,sp3_[O-]P,sp3_OP,sp3_C[Si],sp3_CC(P)P,sp3_c[Si](C)(C)C,sp3_c[C@@H](C)O
0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5743,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5745,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig
import os

# Define the feature definition file
fdefName = os.path.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')

# Build the feature factory
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

# Initialize a dictionary to store feature counts
feature_counts = {}

def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # Get the features for the molecule
    feats = factory.GetFeaturesForMol(mol)

    feature_count = {}
    for feat in feats:
        # feat_type = feat.GetFamily()
        feat_type = feat.GetType()  # Use the feature type (or type)
        if feat_type in feature_count:
            feature_count[feat_type] += 1
        else:
            feature_count[feat_type] = 1

    return feature_count

# Iterate through each SMILES in the DataFrame
for index, row in filtered_full.iterrows():
    smiles = row['SMILES']
    feature_count = extract_features(smiles)
    feature_counts[index] = feature_count

# Create a DataFrame from the feature counts dictionary and fill missing values with 0
result_data = []
for index, counts in feature_counts.items():
    counts['SMILES'] = filtered_full.loc[index, 'SMILES']
    result_data.append(counts)

result_df_features = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')

# Rename columns with a prefix
result_df_features.columns = ['feature_' + column for column in result_df_features.columns]

# Print the resulting DataFrame
print(result_df_features)


      feature_SingleAtomDonor  feature_SingleAtomAcceptor  feature_Arom5  \
0                         3.0                         5.0            2.0   
1                         1.0                         2.0            1.0   
2                         1.0                         4.0            1.0   
3                         1.0                         5.0            1.0   
4                         0.0                         3.0            0.0   
...                       ...                         ...            ...   
5742                      3.0                         1.0            0.0   
5743                      2.0                         4.0            1.0   
5744                      2.0                         6.0            2.0   
5745                      2.0                         5.0            0.0   
5746                      3.0                         4.0            1.0   

      feature_Arom6  feature_ThreeWayAttach  feature_RH6_6  \
0               1.0      

In [ ]:
Data = pd.concat([filtered_full, full_Descrs,mord_desc_df,result_df_sp3, result_df_non_ring_sp3, result_df_sp2, result_df_non_ring_sp2, result_df_sp, prop_df, result_df_features], axis=1).drop(columns=['Molecule', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])
Data["Molecular_PolarSurfaceArea"] = np.where(pd.isna(Data["Molecular_PolarSurfaceArea"]), Data["TPSA"], Data["Molecular_PolarSurfaceArea"])

In [ ]:
Data['beyond_Lipinski'] = 0

Data.loc[Data['Molecular_Weight'] >= 500, 'beyond_Lipinski'] += 1
Data.loc[Data['LogD'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_H_Donors'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_H_Acceptors'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_RotatableBonds'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[(Data['Num_H_Donors'] + Data['Num_H_Acceptors']) >= 12, 'beyond_Lipinski'] += 1
Data.loc[Data['TPSA'] >= 140, 'beyond_Lipinski'] += 1
Data.loc[Data['FractionCSP3'] <= 0.3, 'beyond_Lipinski'] += 1
Data.loc[Data['NumAromaticRings'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['NOCount'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[Data['NHOHCount'] >= 5, 'beyond_Lipinski'] += 1

In [ ]:
# Specify the file path where you want to save the pickle file
pickle_file_path = "/content/drive/MyDrive/dacon_data_0911.pkl"

# Save the DataFrame as a pickle file
Data.to_pickle(pickle_file_path)

# Print a message to confirm that the file has been saved
print(f"DataFrame saved as {pickle_file_path}")

NameError: ignored

In [5]:
import pandas as pd

# 지정된 pickle 파일 경로
pickle_file_path = "/content/drive/MyDrive/dacon_data_0911.pkl"

# pickle 파일을 DataFrame으로 로드
Data = pd.read_pickle(pickle_file_path)

# 로드된 DataFrame 출력 또는 사용
print(Data)

                 id                                             SMILES  \
0        TRAIN_0000    CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC   
1        TRAIN_0001               Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1   
2        TRAIN_0002                   CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1   
3        TRAIN_0003  Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...   
4        TRAIN_0004                Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2   
...             ...                                                ...   
5742  CHEMBL3810411  Cc1cc(CCC#N)cc(C)c1Oc1cc(Nc2ccc(C#N)cc2)c(N)cc...   
5743  CHEMBL3092907   Cc1nc(N2CCN(C)CC2)c2nc(-c3ccccc3Cl)n(CCO)c2n1.Cl   
5744  CHEMBL2177757  O=C(Cc1ccccc1)Nc1nnc(CCSCCc2nnc(NC(=O)Cc3ccccc...   
5745  CHEMBL3735279       CCOC(=O)c1nc(C)c2c(c1N)C(=O)N(Cc1ccccc1)C2=O   
5746  CHEMBL3353541  C[C@]1(C(=O)N(CCCC(=O)O)Cc2cccc(Cl)c2)CCN1C(=O...   

         MLM     HLM  AlogP  Molecular_Weight  Num_H_Acceptors  Num_H_Donors  \
0     26.010  50.680  3.259    

In [6]:
data_column = [column for column in Data.columns]

In [ ]:
Desc_feature= ['MaxAbsEStateIndex','MaxEStateIndex','MinAbsEStateIndex','MinEStateIndex','qed','HeavyAtomMolWt','NumValenceElectrons',
 'NumRadicalElectrons','MaxPartialCharge','MinPartialCharge','MaxAbsPartialCharge','MinAbsPartialCharge',
 'FpDensityMorgan1','FpDensityMorgan2','FpDensityMorgan3','HallKierAlpha','Ipc','NHOHCount','NOCount']

In [8]:
# 'id' 컬럼에 'TEST' 문자열이 들어가는 행을 testset으로 선택
test = Data[Data['id'].str.contains('TEST')]
print(len(test))
# 'id' 컬럼에 'TEST' 문자열이 들어가지 않는 행을 trainset으로 선택
train = Data[~Data['id'].str.contains('TEST')]
print(len(train))

483
5264


In [9]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# 숫자형식 컬럼들의 min-max 정규화
scaler = MinMaxScaler()
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()
non_features = ['id', 'SMILES', 'MLM', 'HLM','Fingerprint']
features = [column for column in train.columns if column not in non_features]
train[features] = scaler.fit_transform(train[features])

test[features] = scaler.transform(test[features])

<ipython-input-9-238fa936b178>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[features] = scaler.fit_transform(train[features])
<ipython-input-9-238fa936b178>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[features] = scaler.transform(test[features])


In [10]:
train_HLM = train.drop(columns=['MLM']).dropna(axis=0)
print(len(train_HLM))
train_MLM = train.drop(columns=['HLM']).dropna(axis=0)
print(len(train_MLM))

4842
4045


In [11]:
def create_encoders(df):
    """
    데이터프레임 (df)의 각 row를 방문하여, 원자, 연결 타입, 연결 스테레오에 대한 LabelEncoder를 만들어주는 함수
    """
    encoder_atom = defaultdict(lambda: len(encoder_atom))
    encoder_bond_type = defaultdict(lambda: len(encoder_bond_type))
    encoder_bond_stereo = defaultdict(lambda: len(encoder_bond_stereo))
    encoder_bond_type_stereo = defaultdict(lambda: len(encoder_bond_type_stereo))

    target = df['SMILES'].values
    total_num = len(target)
    for i, smiles in enumerate(target):
        print(f'Creating the label encoders for atoms, bond_type, and bond_stereo ... [{i + 1} / {total_num}] done !',
              end='\r')
        m = Chem.MolFromSmiles(smiles)
        m = Chem.AddHs(m)

        for atom in m.GetAtoms():
            encoder_atom[atom.GetAtomicNum()]

        for bond in m.GetBonds():
            encoder_bond_type[bond.GetBondTypeAsDouble()]
            encoder_bond_stereo[bond.GetStereo()]
            encoder_bond_type_stereo[(bond.GetBondTypeAsDouble(), bond.GetStereo())]

    return encoder_atom, encoder_bond_type, encoder_bond_stereo, encoder_bond_type_stereo

def row2data(row, encoder_atom, encoder_bond_type, encoder_bond_stereo, encoder_bond_type_stereo):
    smiles = row.SMILES
    y = row.y

    m = Chem.MolFromSmiles(smiles)
    m = Chem.AddHs(m)

    # Creating node feature vector
    num_nodes = len(list(m.GetAtoms()))
    x = np.zeros((num_nodes, len(encoder_atom.keys())))
    for i in m.GetAtoms():
        x[i.GetIdx(), encoder_atom[i.GetAtomicNum()]] = 1

    x = torch.from_numpy(x).float()

    # Creating edge_index and edge_type
    i = 0
    num_edges = 2 * len(list(m.GetBonds()))
    edge_index = np.zeros((2, num_edges), dtype=np.int64)
    edge_type = np.zeros((num_edges,), dtype=np.int64)
    for edge in m.GetBonds():
        # Getting bond information
        u = min(edge.GetBeginAtomIdx(), edge.GetEndAtomIdx())
        v = max(edge.GetBeginAtomIdx(), edge.GetEndAtomIdx())
        bond_type = edge.GetBondTypeAsDouble()
        bond_stereo = edge.GetStereo()
        bond_label = encoder_bond_type_stereo[(bond_type, bond_stereo)]

        # Storing information
        edge_index[0, i] = u
        edge_index[1, i] = v
        edge_index[0, i + 1] = v
        edge_index[1, i + 1] = u
        edge_type[i] = bond_label
        edge_type[i + 1] = bond_label
        i += 2

    edge_index = torch.from_numpy(edge_index)
    edge_type = torch.from_numpy(edge_type)

    # Creating y
    y = torch.tensor([y]).float()

    # Wrapping all together
    data = Data(
        x=x,
        edge_index=edge_index,
        edge_type=edge_type,
        y=y,
        uid=row.id
    )

    return data

In [12]:
!pip install torch_geometric

In [13]:
!pip install pytorch-tabnet

In [19]:
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_smiles
from torch_geometric.data import Data
from collections import defaultdict
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import RGCNConv, global_add_pool
from torch_geometric.data import InMemoryDataset
from torch_geometric.loader import DataLoader

'''
graph data (x, edge_index) / tabular data (without "smiles") / mlm / hlm
'''

class MultiDataset(Dataset):
    def __init__(self):
        super(MultiDataset, self).__init__()

        self.train_df = train_HLM
        self.train_df.fillna(0, inplace=True)
        self.graph_list = self.row2data(self.train_df)

        self.target_hlm = torch.tensor(self.train_df['HLM'].values.astype(np.float32))

        self.wo_smiles_df = self.train_df.drop(columns=['SMILES', 'id', 'HLM'])
        self.wo_smiles_df = torch.tensor(self.wo_smiles_df.values.astype(np.float32))

    def row2data(self,df):

        encoder_atom = defaultdict(lambda: len(encoder_atom))
        encoder_bond_type = defaultdict(lambda: len(encoder_bond_type))
        encoder_bond_stereo = defaultdict(lambda: len(encoder_bond_stereo))
        encoder_bond_type_stereo = defaultdict(lambda: len(encoder_bond_type_stereo))

        target = df['SMILES'].values
        total_num = len(target)
        for i, smiles in enumerate(target):
            print(f'Creating the label encoders for atoms, bond_type, and bond_stereo ... [{i + 1} / {total_num}] done !',
                  end='\r')
            m = Chem.MolFromSmiles(smiles)
            m = Chem.AddHs(m)

            for atom in m.GetAtoms():
                encoder_atom[atom.GetAtomicNum()]

            for bond in m.GetBonds():
                encoder_bond_type[bond.GetBondTypeAsDouble()]
                encoder_bond_stereo[bond.GetStereo()]
                encoder_bond_type_stereo[(bond.GetBondTypeAsDouble(), bond.GetStereo())]

        graph_list=[]
        for i, row in df.iterrows():
          print(f'Converting each data into torch.Data ... [{i+1} / {len(train_HLM)}] done !', end='\r')
          smiles = row.SMILES
          y = row.HLM

          m = Chem.MolFromSmiles(smiles)
          m = Chem.AddHs(m)

          # Creating node feature vector
          num_nodes = len(list(m.GetAtoms()))
          x = np.zeros((num_nodes, len(encoder_atom.keys())))
          for i in m.GetAtoms():
              x[i.GetIdx(), encoder_atom[i.GetAtomicNum()]] = 1

          x = torch.from_numpy(x).float()

          # Creating edge_index and edge_type
          i = 0
          num_edges = 2 * len(list(m.GetBonds()))
          edge_index = np.zeros((2, num_edges), dtype=np.int64)
          edge_type = np.zeros((num_edges,), dtype=np.int64)
          for edge in m.GetBonds():
              # Getting bond information
              u = min(edge.GetBeginAtomIdx(), edge.GetEndAtomIdx())
              v = max(edge.GetBeginAtomIdx(), edge.GetEndAtomIdx())
              bond_type = edge.GetBondTypeAsDouble()
              bond_stereo = edge.GetStereo()
              bond_label = encoder_bond_type_stereo[(bond_type, bond_stereo)]

              # Storing information
              edge_index[0, i] = u
              edge_index[1, i] = v
              edge_index[0, i + 1] = v
              edge_index[1, i + 1] = u
              edge_type[i] = bond_label
              edge_type[i + 1] = bond_label
              i += 2

          edge_index = torch.from_numpy(edge_index)
          edge_type = torch.from_numpy(edge_type)

          # Creating y
          y = torch.tensor([y]).float()

          # Wrapping all together
          data = Data(
              x=x,
              edge_index=edge_index,
              edge_type=edge_type,
              y=y,
              uid=row.id
          )

          graph_list.append(data)
        print('Complete!')
        return graph_list

    def __getitem__(self, idx):
        return self.graph_list[idx], self.wo_smiles_df[idx], self.target_hlm[idx]

    def __len__(self):
        return len(self.graph_list)

In [20]:
dataset = MultiDataset()

Complete!


In [21]:
HLM_loader = DataLoader(dataset, batch_size=128)

print(len(dataset))

4842


In [36]:
from pytorch_tabnet.tab_network import AttentiveTransformer, FeatTransformer

class TabNetEncoder(nn.Module):
    def __init__(
        self,
        input_dim,
        output_dim,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2,
        epsilon=1e-15,
        virtual_batch_size=256,
        momentum=0.02,
        mask_type="sparsemax",
        group_attention_matrix=None,
        device='cpu'
    ):

        super(TabNetEncoder, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.is_multi_task = isinstance(output_dim, list)
        self.n_d = n_d
        self.n_a = n_a
        self.n_steps = n_steps
        self.gamma = gamma
        self.epsilon = epsilon
        self.n_independent = n_independent
        self.n_shared = n_shared
        self.virtual_batch_size = virtual_batch_size
        self.mask_type = mask_type
        self.initial_bn = nn.BatchNorm1d(self.input_dim, momentum=0.01)
        self.group_attention_matrix = group_attention_matrix
        self.device = device

        if self.group_attention_matrix is None:
            # no groups
            self.group_attention_matrix = torch.eye(self.input_dim).to(self.device)
            self.attention_dim = self.input_dim
        else:
            self.attention_dim = self.group_attention_matrix.shape[0]

        if self.n_shared > 0:
            shared_feat_transform = nn.ModuleList()
            for i in range(self.n_shared):
                if i == 0:
                    shared_feat_transform.append(
                        nn.Linear(self.input_dim, 2 * (n_d + n_a), bias=False)
                    )
                else:
                    shared_feat_transform.append(
                        nn.Linear(n_d + n_a, 2 * (n_d + n_a), bias=False)
                    )

        else:
            shared_feat_transform = None

        self.initial_splitter = FeatTransformer(
            self.input_dim,
            n_d + n_a,
            shared_feat_transform,
            n_glu_independent=self.n_independent,
            virtual_batch_size=self.virtual_batch_size,
            momentum=momentum,
        )

        self.feat_transformers = nn.ModuleList()
        self.att_transformers = nn.ModuleList()

        for step in range(n_steps):
            transformer = FeatTransformer(
                self.input_dim,
                n_d + n_a,
                shared_feat_transform,
                n_glu_independent=self.n_independent,
                virtual_batch_size=self.virtual_batch_size,
                momentum=momentum,
            )
            attention = AttentiveTransformer(
                n_a,
                self.attention_dim,
                group_matrix=group_attention_matrix,
                virtual_batch_size=self.virtual_batch_size,
                momentum=momentum,
                mask_type=self.mask_type,
            )
            self.feat_transformers.append(transformer)
            self.att_transformers.append(attention)

    def forward(self, x, prior=None):
        x = self.initial_bn(x)

        bs = x.shape[0]  # batch size
        if prior is None:
            prior = torch.ones((bs, self.attention_dim)).to(x.device)

        M_loss = 0
        att = self.initial_splitter(x)[:, self.n_d :]
        steps_output = []
        for step in range(self.n_steps):
            M = self.att_transformers[step](prior, att)
            M_loss += torch.mean(
                torch.sum(torch.mul(M, torch.log(M + self.epsilon)), dim=1)
            )
            # update prior
            prior = torch.mul(self.gamma - M, prior)
            # output
            M_feature_level = torch.matmul(M, self.group_attention_matrix)
            masked_x = torch.mul(M_feature_level, x)
            out = self.feat_transformers[step](masked_x)
            d = nn.ReLU()(out[:, : self.n_d])
            steps_output.append(d)
            # update attention
            att = out[:, self.n_d :]

        M_loss /= self.n_steps
        return steps_output, M_loss

    def forward_masks(self, x):
        x = self.initial_bn(x)
        bs = x.shape[0]  # batch size
        prior = torch.ones((bs, self.attention_dim)).to(x.device)
        M_explain = torch.zeros(x.shape).to(x.device)
        att = self.initial_splitter(x)[:, self.n_d :]
        masks = {}

        for step in range(self.n_steps):
            M = self.att_transformers[step](prior, att)
            M_feature_level = torch.matmul(M, self.group_attention_matrix)
            masks[step] = M_feature_level
            # update prior
            prior = torch.mul(self.gamma - M, prior)
            # output
            masked_x = torch.mul(M_feature_level, x)
            out = self.feat_transformers[step](masked_x)
            d = nn.ReLU()(out[:, : self.n_d])
            # explain
            step_importance = torch.sum(d, dim=1)
            M_explain += torch.mul(M_feature_level, step_importance.unsqueeze(dim=1))
            # update attention
            att = out[:, self.n_d :]

        return M_explain, masks

In [23]:

num_node_features = 10  # 원자개수 (RGCN의 입력 노드 특징 벡터의 차원)
node_embedding_dim = 128    # 각 원자들을 나타내는 임베딩 벡터의 차원 수
hidden_channels = (256, 256, 256, 256, 256, 256, 256, 256)  # 각 RGCN 레이어가 만드는 hidden state 벡터의 차원 수
hidden_dims = (1024, 512)  # 각 MLP 레이어가 만드는 hidden state 벡터의 차원 수
dropout = 0.3

##########################################################################################################################


class NodeEncoder(nn.Module):
    def __init__(self, input_dim, embedding_dim):
        super(NodeEncoder, self).__init__()
        self.encoder = nn.Linear(input_dim, embedding_dim)

    def forward(self, x):
        return self.encoder(x)

class GraphFeature(torch.nn.Module):
    def __init__(self, hidden_channels, hidden_dims, num_node_features=10, node_embedding_dim=128, dropout=0.3):
        super(GraphFeature, self).__init__()
        self.node_encoder = NodeEncoder(num_node_features, node_embedding_dim)
        self.prelu = nn.PReLU()
        self.dropout = nn.Dropout(dropout)

        # GCN
        self.conv_layers = nn.ModuleList()
        for i in range(len(hidden_channels)):
            if i == 0:
                conv = RGCNConv(node_embedding_dim, hidden_channels[i], num_relations=6, aggr='add')
            else:
                conv = RGCNConv(hidden_channels[i - 1], hidden_channels[i], num_relations=6, aggr='add')
            self.conv_layers.append(conv)

        self.graph_pooling = nn.Sequential(
            nn.Linear(hidden_channels[-1], hidden_channels[-1]),
            nn.ReLU(),
            nn.Linear(hidden_channels[-1], hidden_channels[-1]),
        )

        # MLP
        self.fc_layers = nn.ModuleList()
        for i in range(len(hidden_dims)):
            if i == 0:
                fc = nn.Linear(hidden_channels[-1], hidden_dims[i])
            else:
                fc = nn.Linear(hidden_dims[i - 1], hidden_dims[i])
            self.fc_layers.append(fc)
        self.out = nn.Linear(hidden_dims[-1], 256)

    def forward(self, x, edge_index, edge_type, batch):
        # 1. Encode each atom to a vector representation
        x = self.node_encoder(x)

        # 2. Obtain node embeddings through gnn
        for i, conv in enumerate(self.conv_layers):
            skip = x
            x = conv(x, edge_index, edge_type)
            x = self.prelu(x + skip)
            x = F.normalize(x, 2, 1)

        # 3. graph pooling
        x = self.graph_pooling(x)
        x = global_add_pool(x, batch)  # [batch_size, hidden_channels]

        # 4. Readout phase
        for i, fc in enumerate(self.fc_layers):
            x = fc(x)
            x = self.dropout(x)
            x = F.relu(x)

        # 5. Final prediction
        x = F.relu(self.out(x))

        return x

In [24]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.2):
        super(MLP, self).__init__()

        # 입력 차원, 은닉층 차원, 출력 차원 설정
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # MLP의 레이어 정의
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # 입력층 -> 은닉층
        self.fc2 = nn.Linear(hidden_dim, output_dim) # 은닉층 -> 출력층

        # 드롭아웃 레이어 (선택적으로 사용)
        self.dropout = nn.Dropout(dropout_prob)

        # 활성화 함수 (예: ReLU)
        self.activation = nn.ReLU()

    def forward(self, x):
        # 입력 데이터 x를 MLP에 통과시키는 forward 함수
        x = self.fc1(x)          # 입력층 -> 은닉층
        x = self.activation(x)   # 활성화 함수 (예: ReLU)
        x = self.dropout(x)      # 드롭아웃 레이어 (선택적으로 사용)
        x = self.fc2(x)          # 은닉층 -> 출력층
        return x


In [56]:
import torch.nn as nn

class GraphTab(nn.Module):
    def __init__(self, graph_dict, tabnet_dict, reg_emb, drop_ratio, out_dim, device='cpu'):
        super(GraphTab, self).__init__()

        self.device = device

        self.graph_feature = GraphFeature(graph_dict['hidden_channels'], graph_dict['hidden_dims'], graph_dict['num_node_features'], graph_dict['node_embedding_dim'],  graph_dict['dropout'] )

        # Define an MLP for tabular data
        self.tabnet_feature = nn.Sequential(
            nn.Linear(tabnet_dict['input_dim'], tabnet_dict['hidden_dim']),
            nn.ReLU(),
            nn.Dropout(p=drop_ratio),
            nn.Linear(tabnet_dict['hidden_dim'], tabnet_dict['hidden_dim']),
            nn.ReLU(),
            nn.Dropout(p=drop_ratio),
            nn.Linear(tabnet_dict['hidden_dim'], out_dim)  # Changed output_dim to out_dim
        )

        self.regressor = nn.Sequential(nn.Linear(257, reg_emb),
                                       nn.BatchNorm1d(reg_emb),
                                       nn.ELU(),
                                       nn.Dropout(p=drop_ratio),
                                       nn.Linear(reg_emb, reg_emb),
                                       nn.BatchNorm1d(reg_emb),
                                       nn.ELU(),
                                       nn.Dropout(p=drop_ratio),
                                       nn.Linear(reg_emb, out_dim))

    def forward(self, node_attr, edge_idx, edge_type, batch, tabular):
        gr_ft = self.graph_feature(node_attr, edge_idx, edge_type, batch) # (batch, embed_dim)

        # Pass tabular data through the MLP
        tab_ft = self.tabnet_feature(tabular) # (batch, out_dim)

        gr_ft.to(self.device)
        tab_ft.to(self.device)

        # Combine the features from the graph and tabular data
        combined_features = torch.cat((gr_ft, tab_ft), dim=1)

        res = self.regressor(combined_features)
        return res


In [58]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_hat, y):
        loss = torch.sqrt(self.mse(y_hat,y))
        return loss

In [59]:

graph_params = {
    'hidden_channels': (128, 128, 128, 128, 128, 128, 128, 128),
    'hidden_dims': (1024, 512),
    'num_node_features' : 10,
    'node_embedding_dim' : 128,
    'dropout': 0.3
}

tabnet_params = {
    'input_dim': 2064,
    'hidden_dim': 256,
    'output_dim': 128,
    'n_da': 256, # output dimension
    'n_steps': 3,
}

model = GraphTab(graph_dict=graph_params,
                 tabnet_dict=tabnet_params,
                 reg_emb=128,
                 drop_ratio=0.1,
                 out_dim=1,
                 device='cuda')

In [61]:
EPOCHS = 100
model.to(device='cuda')

criterion = RMSELoss().to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

losses = []

model.train()
for ep in range(EPOCHS):
    ep_loss = 0
    for gr, tr_df, hlm_target in HLM_loader:
        gr_x = gr.x.type(torch.float32).to('cuda')
        gr_edge_idx = gr.edge_index.to('cuda')
        gr_edge_type = gr.edge_type.to('cuda')
        gr_batch = gr.batch.to('cuda')

        tabular = tr_df.to('cuda')
        hlm_target = hlm_target.view(-1, 1).to('cuda')

        optimizer.zero_grad()
        predict = model(gr_x, gr_edge_idx, gr_edge_type, gr_batch, tabular)

        loss = criterion(predict, hlm_target)
        loss.backward()
        ep_loss = loss.item()

        optimizer.step()
    losses.append(ep_loss)

    print(f'Epoch: {ep+1} / rmse: {ep_loss}')

Epoch: 1 / rmse: 63.87650680541992
Epoch: 2 / rmse: 60.33805847167969
Epoch: 3 / rmse: 54.525691986083984
Epoch: 4 / rmse: 47.14356231689453
Epoch: 5 / rmse: 38.41265106201172
Epoch: 6 / rmse: 34.55935287475586
Epoch: 7 / rmse: 32.82981872558594
Epoch: 8 / rmse: 29.90688133239746
Epoch: 9 / rmse: 28.74506950378418
Epoch: 10 / rmse: 30.83424949645996
Epoch: 11 / rmse: 28.803401947021484
Epoch: 12 / rmse: 27.15066146850586
Epoch: 13 / rmse: 26.07340431213379
Epoch: 14 / rmse: 30.290393829345703
Epoch: 15 / rmse: 26.391637802124023
Epoch: 16 / rmse: 28.317848205566406
Epoch: 17 / rmse: 26.07669448852539
Epoch: 18 / rmse: 26.15704917907715
Epoch: 19 / rmse: 26.84959602355957
Epoch: 20 / rmse: 25.865419387817383
Epoch: 21 / rmse: 26.114431381225586
Epoch: 22 / rmse: 25.04412078857422
Epoch: 23 / rmse: 25.291015625
Epoch: 24 / rmse: 25.329193115234375
Epoch: 25 / rmse: 25.19713020324707
Epoch: 26 / rmse: 25.901836395263672
Epoch: 27 / rmse: 25.899169921875
Epoch: 28 / rmse: 25.41651153564453

KeyboardInterrupt: ignored

# Dacon 코드랑 기존코드 참고해서 모델 이어가기

In [ ]:
df_submission

,id,MLM,HLM
0,TEST_000,22.072287,49.431904
1,TEST_001,67.795456,80.603355
2,TEST_002,48.650043,63.674030
3,TEST_003,49.560764,69.864143
4,TEST_004,60.214588,75.319855
...,...,...,...
478,TEST_478,6.380654,25.069542
479,TEST_479,72.472137,86.032578
480,TEST_480,38.472294,64.516289
481,TEST_481,44.892677,71.460587
